# Outline
- Generate training and testing set
- Train neural network
- Test neural network
- Analyze results

In [1]:
# Import necessary libraries
import graphcalc as gc
import spektral as sp
from spektral.data import Dataset, Graph, Loader
import networkx as nx
import random
import numpy as np

In [9]:
NUM_GRAPHS = 2000
MAX_NODES = 64
SPLIT_INDEX = int(0.8 * NUM_GRAPHS)

In [3]:
# Dataset class
class DominationGraphsDataset(Dataset):
    """
    Spektral Dataset that holds `num_graphs` random graphs with labels equal to
    their domination numbers.
    """

    def __init__(self, num_graphs=NUM_GRAPHS, max_nodes=MAX_NODES, **kwargs):
        if max_nodes < 10:
            raise ValueError("`max_nodes` must be at least 10.")
        self.num_graphs = num_graphs
        self.max_nodes = min(max_nodes, 64)  # Enforce 64-node ceiling
        super().__init__(**kwargs)

    # --------------------------------------------------------------------- #
    #  Required by Spektral: returns a list of Graph objects                #
    # --------------------------------------------------------------------- #
    def read(self):
        graphs = []

        for _ in range(self.num_graphs):
            n = random.randint(10, self.max_nodes)

            # Change p’s range to control sparsity / density if desired
            p = random.uniform(0.1, 0.5)
            G_nx = nx.gnp_random_graph(n, p)

            # Exact domination number (NP-hard; fine for ≤64-node graphs)
            dom_num = gc.domination_number(G_nx)

            # Convert to numpy adjacency matrix
            a = nx.to_numpy_array(G_nx, dtype=np.float32)

            # Simple identity node features (n × n)
            x = np.eye(n, dtype=np.float32)

            # Store domination number as a 1-element label vector
            y = np.array([dom_num], dtype=np.float32)

            graphs.append(Graph(x=x, a=a, y=y))

        return graphs

In [ ]:
# Instantiate the dataset
ds = DominationGraphsDataset()

In [7]:
# Check the dataset
g0 = ds[0]            # Spektral Graph instance

print("Domination number:", int(g0.y))      # scalar label
print("Adjacency matrix shape:", g0.a.shape)
print("Adjacency matrix (dense):")
print(g0.a.astype(int))                     # cast to int for readability

print("Node-feature matrix shape:", g0.x.shape)


Domination number: 7
Adjacency matrix shape: (61, 61)
Adjacency matrix (dense):
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 1 0]]
Node-feature matrix shape: (61, 61)


C:\Users\kylej\AppData\Local\Temp\ipykernel_33952\856224493.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Domination number:", int(g0.y))      # scalar label


In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

class GNN(Model):
    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out 

In [12]:
model = GNN(32, 64)
model.compile('adam', 'categorical_crossentropy')

In [13]:
ds_train = ds[:SPLIT_INDEX]
ds_test = ds[SPLIT_INDEX:]

In [14]:
from spektral.data import BatchLoader

loader = BatchLoader(ds_train, batch_size=32)

In [15]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

ValueError: Unrecognized data type: x=<spektral.data.loaders.BatchLoader object at 0x00000239D6383FB0> (of type <class 'spektral.data.loaders.BatchLoader'>)